This notebook allows you to generate different maps showing the number of associations per area.

The packages requirements can be infered from the `Imports` section.

You will need 3 files to run the code. You can modify the path of those files below.

- the data cleaned by Brad `clean_data_small.csv`
- the shape of the area of the postal codes `codes_postaux_region.shp`. It can be found here : https://www.data.gouv.fr/fr/datasets/fond-de-carte-des-codes-postaux/
- the shape of the departements `departements-version-simplifiee.geojson`. It can be found here : https://github.com/gregoiredavid/france-geojson

It is assumed that you have a folder `../figures` where the html files of the interactive maps will be saved.

In [ ]:
RAW_CSV = '../data/raw/clean_data_small.csv'
CP_SHP = '../../code_postaux/codes_postaux_region.shp'
DEP_GEOJSON = '../../france-geojson/departements-version-simplifiee.geojson'

# Imports

In [1]:
import numpy as np
import pandas as pd
import re

In [4]:
import json

import bokeh
from bokeh.models import (HoverTool, LinearColorMapper, BasicTicker, 
                          ColorBar, ColumnDataSource, Legend)
from bokeh.plotting import figure, output_file, save, show
from bokeh.palettes import Viridis256, OrRd, Spectral6, Reds, Greens, Blues
from bokeh.layouts import row, column
from bokeh.transform import factor_cmap

import geopandas as gpd

#import folium #fond de carte openstreetmap

# Utility functions

In [7]:
def getXYCoords(geometry, coord_type):
    """ Return either x or y coordinates from geometry coordinate sequence. 
    
    Arguments:
    ----------
    geometry: list of tuples of floats
        List for geographic coordinates.
    coord_type: string
        "x" for latitude.
        "y" for longitude.
    
    Returns:
    -------
    coords: list of floats
        List of either latitudes or longitudes extracted from the list of coordinates.
    """
    if coord_type == 'x':
        return [x[0] for x in geometry]
    elif coord_type == 'y':
        return [x[1] for x in geometry]

In [8]:
def getPolyCoords(geometry, coord_type):
    """ Extract a list of latitudes or longitudes from a Polygon shape.
    
    Arguments:
    ----------
    geometry: shapely.geometry.Polygon or Polygon-like dict
        Definition of a polygon by its coordinates.
    coord_type: string
        "x" for latitude.
        "y" for longitude.
        
    Returns:
    --------
    coords: list of floats
        List of either latitudes or longitudes extracted from the Polygon.
    """
    ext = geometry['coordinates'][0]
    return getXYCoords(ext, coord_type)

In [9]:
def multiGeomHandler(multi_geometry, coord_type):
    """ Function for handling MultiPolygon.
    
    Bokeh seems to handle multi-geometries since October 30, but no documentation on this matter could be found.
    We use only the polygon of the multi-geometry with the most entries, which is enough for us, as we do not 
    deal with archipelagos.
    
    Arguments:
    ----------
    multi_geometry: shapely.geometry.MultiPolygon or MultiPolygon-like dict
        Definition of a multi-polygon by its coordinates.
    coord_type: string
        "x" for latitude.
        "y" for longitude.
        
    Returns:
    --------
    coords: list of floats
        List of either latitudes or longitudes extracted from the MultiPolygon.
    """

    for i, part in enumerate(multi_geometry["coordinates"]):
        # On the first part of the Multi-geometry initialize the coord_array (np.array)
        if i == 0:
            coord_arrays = getXYCoords(part[0], coord_type)
            max_length = len(part[0])
        else:
            if len(part[0]) > max_length:
                max_length = len(part[0])
                coord_arrays = getXYCoords(part[0], coord_type)
    # Return the coordinates
    return coord_arrays

In [10]:
def getCoords(row, geom_col, coord_type):
    """ Returns coordinates ('x' or 'y') of a geometry Polygon or a multi-geometry MultiPolygon as a list.
    
    Arguments:
    ----------
    row: dictionary
        Feature containing the geometry
    geom_col: string
        Name of the entry of `row` that contains the geometry
    coord_type: string
        "x" for latitude.
        "y" for longitude.
    
    Returns:
    --------
    coords: list of floats
        List of either latitudes or longitudes extracted from the MultiPolygon.
    """
    # Get geometry
    geom = row[geom_col]

    # Check the geometry type
    gtype = geom["type"]

    if gtype == "Polygon":
        return list( getPolyCoords(geom, coord_type) )

    # Multi geometries
    # ----------------
    else:
        return list( multiGeomHandler(geom, coord_type) )

In [47]:
def create_args_for_maps(df, area_name, column_name):
    """ Create the arguments for the interactive maps for one type of area
    
    Arguments:
    ----------
    area_name: string
        Full name of the area type
    column_name: string
        Name of the column encoding the area
        
    Returns:
    --------
    arguments: dictionary
        Dictionary to use as datasource in the maps.
    """
    if area_name == 'cp':
        geojson_file_name = "../../code_postaux/codes_postaux_region.shp"
        id_col = 'ID'
        cols = [id_col, 'geometry']
    elif area_name == 'dep':
        geojson_file_name = "../../france-geojson/departements-version-simplifiee.geojson"
        id_col = 'code'
        cols = [id_col, 'geometry', 'nom']
    # create dataframe
    data = gpd.read_file(geojson_file_name, encoding='utf-8')[cols]
    joined_df = data.merge(df, left_on=id_col, right_on=column_name, how="left")
    joined_df.drop(columns={"geometry", id_col}, inplace=True)
    
    # get x and y for contour of the areas
    if area_name == 'cp':
        borders = eval(data.to_json())
    elif area_name == 'dep':
        borders = json.load(open(geojson_file_name))
    if 'features' in borders:
        areas_x = [getCoords(i, geom_col="geometry", coord_type="x") for i in borders['features']]
        areas_y = [getCoords(i, geom_col="geometry", coord_type="y") for i in borders['features']]
    else:
        areas_x = [getCoords(borders, geom_col='geometry', coord_type="x")]
        areas_y = [getCoords(borders, geom_col='geometry', coord_type="y")]
    
    # create dict for arguments
    arguments = {"x": areas_x, "y": areas_y,}
    for column in joined_df.columns:
         arguments["{}".format(column)] = joined_df[column].values
    return arguments

In [12]:
def color_helpers(low, high, title, palette):
    """Define the color-related items for a map
    
    Arguments:
    ----------
    low: float
        Lower bound for the values to encode with the colors
    high: float
        Upper bound for the values to encode with the colors
    title: string
        Title for the color bar
    
    Returns:
    --------
    color_mapper: bokeh.plotting.LinearColorMapper
        Mapper value -> color for the map
    color_bar_plot: bokeh.plotting.Figure
        Block displaying a color bar with a title
    """
    color_mapper = LinearColorMapper(palette=list(reversed(palette)), low=low, high=high)
    color_bar = ColorBar(color_mapper=color_mapper, major_label_text_align='left',ticker=BasicTicker(), location=(0, 0))
    color_bar_plot = figure(title=title, title_location="right",
                        toolbar_location=None, min_border=0, width=130, height=450,
                        outline_line_color=None)

    color_bar_plot.add_layout(color_bar, 'left')
    color_bar_plot.title.align="center"
    color_bar_plot.title.text_font_size = '12pt'
    return color_mapper, color_bar_plot

In [13]:
def color_helpers_cat(categories, palette):
    color_mapper = factor_cmap('Réponses', palette=palette, factors=categories)['transform']
    return color_mapper, None 

In [14]:
def create_map_figure(source, color_mapper, value_column, value_name, height=450, width=450):
    """Create the figure for a map
    
    Arguments:
    ----------
    source: bokeh.models.ColumnDataSource
        Datasource for the map
    color_mapper: bokeh.plotting.LinearColorMapper
        Mapper value -> color for the map
    value_name: string
        Display name of the value to be displayed
        
    Returns:
    --------
    p: bokeh.plotting.Figure
        Figure containing a map
    """
    TOOLS = 'hover,wheel_zoom,box_zoom,reset,pan'#"pan,hover"
    p = figure(
        #title=value_name,
        tools=TOOLS,
        #toolbar_location=None,
        x_axis_location=None, y_axis_location=None,
        width=width, height=height
    )
    p.grid.grid_line_color = None
    p.patches('x', 'y', source=source,
              #fill_color='values',
              fill_color={'field': value_column, 'transform': color_mapper},
              fill_alpha=0.8, line_color="black", line_width=0.3)
    return p

In [15]:
def define_hover(p, initial_values):
    """ Define the hover behavior for a map
    
    Arguments:
    ----------
    p: bokeh.plotting.Figure
        Figure containing a map
    area_name: string
        Name of the subdivision of the French territory
    display_name: string
        Name of the quantity to display in the hover
    
    Returns:
    --------
    hover: bokeh.models.HoverTool
        Hover item.
    """
    hover = p.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    tooltips = [(name, '@%s{1}' % cols) 
                for name, cols in zip(initial_values['hover_names'], 
                                      initial_values['hover_columns'])]
    if initial_values['zone_code'] == 'cp':
        zone_code = "@CP"
    elif initial_values['zone_code'] == 'dep':
        zone_code = "@nom"
    hover.tooltips = [(initial_values['zone_name'], zone_code)] + tooltips
    return hover

In [16]:
def create_map(initial_source, initial_values, height, width):
    """ Create a map
    
    Arguments:
    ----------
    initial_source: dictionary of ColumnDataSource
        Available sources for the map
    initial_values: dictionary
        Initial values for the map
    height, width: int
        Size of the map
    
    Returns:
    --------
    map_figure: bokeh.plotting.Figure
        Figure containing a map
    hover: bokeh.models.HoverTool
        Hover item.
    cmap: bokeh.plotting.LinearColorMapper
        Mapper value -> color for the map
    color_bar_plot: bokeh.plotting.Figure
        Block displaying a color bar with a title
    """
    
    # color handling
    if 'categories' in initial_values:
        cmap, color_bar_plot = color_helpers_cat(
            initial_values['categories'], initial_values['palette']
        )
    else:
        cmap, color_bar_plot = color_helpers(
            initial_values['low'], initial_values['high'], 
            initial_values['colorbar_title'], initial_values['palette']
        )
    map_figure = create_map_figure(initial_source, cmap, initial_values['value_column'],
                                   initial_values['value_name'], height, width)
    # hover behavior
    hover = define_hover(map_figure, initial_values)
    
    return map_figure, hover, cmap, color_bar_plot

In [71]:
def safe_postal_code(x):
    try:
        return "{:05d}".format(int(x))
    except:
        try:
            return "{:05d}".format(int(float(x)))
        except:
            return np.nan

# Working on the raw dataset

Import the small dataset of Brad

In [66]:
df_raw = pd.read_csv('../data/raw/clean_data_small.csv')
df = df_raw.copy()
df.head()

/home/myriam/DataScience/grand-debat/.venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,date_creat,nature,groupement,titre,titre_court,objet,objet_social1,adrg_declarant,adrg_complemid,adrg_complemgeo,adrg_libvoie,adrg_distrib,adrg_codepostal,adrg_achemine,adrg_pays,objet_social1_desc
0,W012000002,1983-10-10,D,S,ARIADNE,ARIADNE,Favoriser le développement d'une compagnie thé...,6070,NaN,NaN,NaN,39 rue Courteline,NaN,69100,villeurbanne,FRANCE,"théâtre, marionnettes, cirque, spectacles de v..."
1,W012000005,2003-09-01,D,S,UNITE FRANCAISE D'INTERVENTION EN CATASTROPHE ...,UNITE FRANCAISE D'INTERVENTION EN ...,Intervenir lors de catastrophes naturelles sur...,20000,NaN,NaN,NaN,18 rue Ampère,NaN,69270,FONTAINE SUR SAONE,FRANCE,"associations caritatives, humanitaires, aide a..."
2,W033002617,2008-05-19,D,S,ASSOCIATION MAYOTTE FOOTBALL CLUB DE BELLERIVE...,MAYOTTE FOOTBALL CLUB BSA,pratiquer et développer le football; organisat...,11075,NaN,NaN,NaN,8 RUE EMMANUEL CHABRIER,NaN,03700,BELLERIVE-SUR-ALLIER,FRANCE,"Football (football, futsal)"
3,W423003687,2008-07-02,D,S,SOUL ALMIGHTY,SOUL ALMIGHTY,promouvoir la musique reggae tout en participa...,6030,NaN,NaN,NaN,36 rue de la Roche du Geai,NaN,42000,Saint-Étienne,FRANCE,"chant choral, musique"
4,W441001349,2012-03-29,D,S,TUMBA MUSIC,TUMBA MUSIC,développer des actions culturelles de soutien ...,6030,NaN,NaN,NaN,6 rue du Boispéan,NaN,44110,Châteaubriant,FRANCE,"chant choral, musique"


Import the postal code files with `départements` and shape of the areas

In [68]:
cp_df = gpd.read_file("../../code_postaux/codes_postaux_region.shp")

Set the correct `départements` of Corsica (`2A` and `2B`)

In [69]:
cp_2A = cp_df[(cp_df['ID'] >= '20000') & (cp_df['ID'] < '20200')]
cp_df.loc[cp_2A.index, 'DEP'] = '2A'
cp_2B = cp_df[(cp_df['ID'] >= '20200') & (cp_df['ID'] < '21000')]
cp_df.loc[cp_2B.index, 'DEP'] = '2B'

In [70]:
cp_df.head()

,ID,LIB,DEP,SURF,POP2010,MEN2010,geometry
0,26140,Saint-Rambert-d'Albon,26,82.710226,12812.0,5148.08200,"POLYGON ((849939.96 6461152.034999999, 848364...."
1,26150,Die,26,315.349961,6301.0,3011.68790,"POLYGON ((892450.0499999999 6403419.96, 891173..."
2,26160,La Bégude-de-Mazenc,26,181.940199,7285.0,3040.83720,"POLYGON ((862423.98 6386618.039999999, 861627...."
3,26300,Bourg-de-Péage,26,236.697761,28064.0,11312.93600,"POLYGON ((873008.0099999999 6428766.989999999,..."
4,26170,Buis-les-Baronnies,26,290.688573,5512.0,2543.62744,"POLYGON ((899364.96 6353489.024999999, 897448...."


In [72]:
df['adrg_codepostal'] = df['adrg_codepostal'].apply(safe_postal_code)

Merge the RNA dataframe with the postal code one

In [75]:
df2 = df.merge(cp_df, right_on='ID', left_on='adrg_codepostal', how='left')

Save number of associations per postal code

In [127]:
df3 = df2.dropna(subset=['ID']).groupby('ID').count()['id']
df3.name = 'count'
df3.index.name = 'CP'
df3.to_csv('../data/interim/20191105_nb_asso_per_cp.csv', header=True)
df3.head()

CP
01000    1886
01090     246
01100     472
01110     219
01120     440
Name: count, dtype: int64

Save number of associations per departements

In [83]:
df4 = df3.reset_index().merge(cp_df, left_on='CP', right_on='ID').drop('ID', axis=1)
df5 = df4[['count', 'DEP']].groupby('DEP').sum()
df5.to_csv('../data/interim/20191105_nb_asso_per_dep.csv')
df5.head()

,count
DEP,
01,16105
02,9266
03,9638
04,5987
05,5480


Save the number of associations per departements for 10000hab

In [131]:
df6 = df4[['DEP', 'POP2010', 'count']].groupby('DEP').sum().reset_index()
df6['count'] = (df6['count'] / df6['POP2010'] * 10000).apply(lambda x: round(x))
df6 = df6.drop('POP2010', axis=1)
df6.to_csv('../data/interim/20191105_nb_asso_per_dep_per10000.csv', index=False)
df6.head()

,DEP,count
0,01,269
1,02,171
2,03,281
3,04,377
4,05,397


Save the number of associations per postal code for 10000hab

In [150]:
df7 = df4[['CP', 'POP2010', 'count']].copy()
df7 = df7[df7['POP2010'] > 0] # Two islands have a zero population
df7['count'] = (df7['count'] / df7['POP2010'] * 10000).apply(lambda x: round(x))
df7 = df7.drop('POP2010', axis=1)
df7 = df7.drop(df7[df7['CP'] == '69125'].index) # Outlier
df7 = df7.drop(df7[df7['CP'] == '72000'].index) # Outlier
df7.to_csv('../data/interim/20191105_nb_asso_per_cp_per10000.csv', index=False)
df7.head()

,CP,count
0,01000,415
1,01090,285
2,01100,140
3,01110,312
4,01120,246


Save the number of associations per postal code in Isère only

In [111]:
df3.head()

CP
01000    1886
01090     246
01100     472
01110     219
01120     440
Name: count, dtype: int64

In [122]:
df8 = df3.reset_index()
df8 = df8[(df8['CP'] >= '38000') & (df8['CP'] < '39000')]
df8 = df8.set_index('CP')
df8.to_csv('../data/interim/20191105_nb_asso_per_cp_isere.csv', header=True)

# Generating different maps

Common settings

In [124]:
size = 1000
initial_values=dict(
        value_name='count',
        value_column='count',
        colorbar_title='Number of associations',
        hover_columns=['count'],
        hover_names=['Count'],
        palette=Blues[9]
    )

Map at the level of postal code

In [138]:
filename = '20191105_nb_asso_per_cp'
df_ = pd.read_csv('../data/interim/%s.csv' % filename, low_memory=False)
df_['CP'] = df_['CP'].apply(lambda x: "{:05d}".format(x))
initial_values['zone_name'] = 'Postal code'
initial_values['zone_code'] = 'cp'
initial_values['low'] = df_['count'].min()
initial_values['high'] = df_['count'].max()
initial_values['colorbar_title'] = 'Number of associations'
args = create_args_for_maps(df_, 'cp', 'CP') #create_args_for_maps(df, area_name, column_name)
datasources = ColumnDataSource(args)
maps, hovers, cmaps, color_bar_plots  = create_map(
    datasources, 
    initial_values,
    size, size)
output_file('../figures/%s.html' % filename)
row_ = row(maps, color_bar_plots)
save(row_)

'/home/myriam/DataScience/dataforgoodgrenoble/figures/20191105_nb_asso_per_cp.html'

Map at the level of postal code, normalize by hab

In [151]:
filename = '20191105_nb_asso_per_cp_per10000'
df_ = pd.read_csv('../data/interim/%s.csv' % filename, low_memory=False)
df_.columns = ['CP', 'count']
df_['CP'] = df_['CP'].apply(lambda x: "{:05d}".format(x))
initial_values['zone_name'] = 'Postal code'
initial_values['zone_code'] = 'cp'
initial_values['low'] = df_['count'].min()
initial_values['high'] = df_['count'].max()
initial_values['colorbar_title'] = 'Number of associations per 10k hab'
args = create_args_for_maps(df_, 'cp', 'CP') #create_args_for_maps(df, area_name, column_name)
datasources = ColumnDataSource(args)
maps, hovers, cmaps, color_bar_plots  = create_map(
    datasources, 
    initial_values,
    size, size)
output_file('../figures/%s.html' % filename)
row_ = row(maps, color_bar_plots)
save(row_)

'/home/myriam/DataScience/dataforgoodgrenoble/figures/20191105_nb_asso_per_cp_per10000.html'

Map by departements

In [154]:
filename = '20191105_nb_asso_per_dep'
df_ = pd.read_csv('../data/interim/%s.csv' % filename, low_memory=False)
initial_values['zone_name'] = 'Departement'
initial_values['zone_code'] = 'dep'
initial_values['low'] = df_['count'].min()
initial_values['high'] = df_['count'].max()
initial_values['colorbar_title'] = 'Number of associations'
args = create_args_for_maps(df_, 'dep', 'DEP') #create_args_for_maps(df, area_name, column_name)
datasources = ColumnDataSource(args)
maps, hovers, cmaps, color_bar_plots  = create_map(
    datasources, 
    initial_values,
    size, size)
output_file('../figures/%s.html' % filename)
row_ = row(maps, color_bar_plots)
save(row_)

'/home/myriam/DataScience/dataforgoodgrenoble/figures/20191105_nb_asso_per_dep.html'

Map by departements per 10k habs

In [153]:
filename = '20191105_nb_asso_per_dep_per10000'
df_ = pd.read_csv('../data/interim/%s.csv' % filename, low_memory=False)
initial_values['zone_name'] = 'Departement'
initial_values['zone_code'] = 'dep'
initial_values['low'] = df_['count'].min()
initial_values['high'] = df_['count'].max()
initial_values['colorbar_title'] = 'Number of associations per 10k hab'
args = create_args_for_maps(df_, 'dep', 'DEP') #create_args_for_maps(df, area_name, column_name)
datasources = ColumnDataSource(args)
maps, hovers, cmaps, color_bar_plots  = create_map(
    datasources, 
    initial_values,
    size, size)
output_file('../figures/%s.html' % filename)
row_ = row(maps, color_bar_plots)
save(row_)

'/home/myriam/DataScience/dataforgoodgrenoble/figures/20191105_nb_asso_per_dep_per10000.html'

Map of Isère

In [155]:
filename = '20191105_nb_asso_per_cp_isere'
df_ = pd.read_csv('../data/interim/%s.csv' % filename, low_memory=False)
df_.columns = ['CP', 'count']
df_['CP'] = df_['CP'].apply(lambda x: "{:05d}".format(x))
initial_values['zone_name'] = 'Postal code'
initial_values['zone_code'] = 'cp'
initial_values['low'] = df_['count'].min()
initial_values['high'] = df_['count'].max()
initial_values['colorbar_title'] = 'Number of associations per 10k hab'
args = create_args_for_maps(df_, 'cp', 'CP') #create_args_for_maps(df, area_name, column_name)
datasources = ColumnDataSource(args)
maps, hovers, cmaps, color_bar_plots  = create_map(
    datasources, 
    initial_values,
    size, size)
output_file('../figures/%s.html' % filename)
row_ = row(maps, color_bar_plots)
save(row_)

'/home/myriam/DataScience/dataforgoodgrenoble/figures/20191105_nb_asso_per_cp_isere.html'